In [62]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# =====================================================================
# STEP 1: Create a neural network from scratch
# =====================================================================
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        # Initialize parameters
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * 0.01
        self.b2 = np.zeros((1, output_size))
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def forward(self, X):
        # Input layer -> Hidden layer
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.relu(self.z1)
        
        # Hidden layer -> Output layer
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def compute_loss(self, y, y_hat):
        # Binary cross-entropy loss
        m = y.shape[0]
        loss = -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
        return loss
    
    def backward(self, X, y, learning_rate):
        m = y.shape[0]
        
        # Output layer gradients
        dz2 = self.a2 - y
        dw2 = (1/m) * np.dot(self.a1.T, dz2)
        db2 = (1/m) * np.sum(dz2, axis=0, keepdims=True)
        
        # Hidden layer gradients
        dz1 = np.dot(dz2, self.W2.T) * (self.z1 > 0)  # ReLU derivative
        dw1 = (1/m) * np.dot(X.T, dz1)
        db1 = (1/m) * np.sum(dz1, axis=0, keepdims=True)
        
        # Update parameters
        self.W2 -= learning_rate * dw2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dw1
        self.b1 -= learning_rate * db1

# =====================================================================
# STEP 2: Create synthetic dataset
# =====================================================================
X, y = make_classification(n_samples=1000, n_features=2, n_classes=2, 
                           n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                   random_state=42)

# =====================================================================
# STEP 3: Train the neural network
# =====================================================================
nn = NeuralNetwork(input_size=2, hidden_size=9, output_size=1)
learning_rate = 0.5
epochs = 1000

for epoch in range(epochs):
    # Forward pass
    y_hat = nn.forward(X_train)
    
    # Compute loss
    loss = nn.compute_loss(y_train.reshape(-1,1), y_hat)
    
    # Backward pass
    nn.backward(X_train, y_train.reshape(-1,1), learning_rate)
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss:.4f}")

# =====================================================================
# STEP 4: Make predictions
# =====================================================================
def predict(X):
    prob = nn.forward(X)
    return (prob > 0.5).astype(int)

# Test accuracy
train_acc = np.mean(predict(X_train) == y_train.reshape(-1,1))
test_acc = np.mean(predict(X_test) == y_test.reshape(-1,1))

print(f"\nTraining Accuracy: {train_acc:.2f}")
print(f"Test Accuracy: {test_acc:.2f}")

Epoch 0, Loss: 0.6932
Epoch 100, Loss: 0.3303
Epoch 200, Loss: 0.3287
Epoch 300, Loss: 0.3273
Epoch 400, Loss: 0.3233
Epoch 500, Loss: 0.3224
Epoch 600, Loss: 0.3221
Epoch 700, Loss: 0.3218
Epoch 800, Loss: 0.3216
Epoch 900, Loss: 0.3215

Training Accuracy: 0.87
Test Accuracy: 0.89
